# Laborato 7.2

In [6]:
import nltk
import numpy as np
from nltk.stem.snowball import SnowballStemmer
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /home/sergio/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/sergio/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /home/sergio/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

## P3- Matriz de similitudes
> Elabore una matriz de similitud de coseno entre los documentos de la colección "El Señor de los Anillos". Debe aplicar los pesos TF-IDF. 

### 1- Preprocesamiento

In [7]:
def preprocesamiento(texto):
	# tokenizar
	tokens = nltk.word_tokenize(texto)
	
	# filtrar stopwords
	stopwords = nltk.corpus.stopwords.words('spanish')
	tokens = [word for word in tokens if word.lower() not in stopwords]
		
	# reducir palabras
	stemmer = SnowballStemmer('spanish')
	tokens = [stemmer.stem(word) for word in tokens]
	
	return ' '.join(tokens)


textos = ["archivo_1.txt", "archivo_2.txt", "archivo_3.txt", "archivo_4.txt", "archivo_5.txt", "archivo_6.txt"]
textos_procesados = []
indice = {}
for file_name in textos:
  file = open('docs2/'+file_name)
  texto = file.read().rstrip()
  texto = preprocesamiento(texto)  
  textos_procesados.append(texto)

### 2- Similitud de coseno

In [8]:
import math
from collections import Counter

def compute_tf(text):
		tf_dict = {}
		tokens = text.split()
		total_tokens = len(tokens)
		token_counts = Counter(tokens)
		for token, count in token_counts.items():
				tf_dict[token] = count / total_tokens
		return tf_dict

def compute_idf(documents):
		idf_dict = {}
		total_docs = len(documents)
		all_tokens_set = set([token for doc in documents for token in doc.split()])
		for token in all_tokens_set:
				containing_docs = sum(1 for doc in documents if token in doc.split())
				idf_dict[token] = math.log(total_docs / (1 + containing_docs))
		return idf_dict

def compute_tfidf(collection):
		tfidf_collection = []
		idf = compute_idf(collection)
		for doc in collection:
				tf = compute_tf(doc)
				tfidf = {token: tf_val * idf[token] for token, tf_val in tf.items()}
				tfidf_collection.append(tfidf)
		return tfidf_collection

def cosine_sim(Q, Doc):  
	# aplicar la similitud de coseno y construir la matriz
	intersection = set(Q.keys()) & set(Doc.keys())
	numerator = sum([Q[x] * Doc[x] for x in intersection])

	sum1 = sum([Q[x]**2 for x in Q.keys()])
	sum2 = sum([Doc[x]**2 for x in Doc.keys()])
	denominator = math.sqrt(sum1) * math.sqrt(sum2)

	if not denominator:
			return 0.0
	else:
			return float(numerator) / denominator
  
textos_tfidf = compute_tfidf(textos_procesados)

matriz = []
for doc1 in textos_tfidf:
  row = []
  for doc2 in textos_tfidf:  
    row.append(cosine_sim(doc1, doc2))
  matriz.append(row)

def print_matriz(matriz):
	for row in matriz:
		print(row)

print_matriz(matriz)


[0.9999999999999989, 0.08276528033635311, 0.11924760421820178, 0.09725448769061039, 0.11469864746977565, 0.152575318863385]
[0.08276528033635311, 1.0000000000000007, 0.10182864026875772, 0.11974650017585393, 0.08466139862686341, 0.09084068775666247]
[0.11924760421820178, 0.10182864026875772, 0.999999999999999, 0.18587237114691205, 0.09600114094278893, 0.16387233403063464]
[0.09725448769061039, 0.11974650017585393, 0.18587237114691205, 0.9999999999999998, 0.1119984153615975, 0.148507501475717]
[0.11469864746977565, 0.08466139862686341, 0.09600114094278893, 0.1119984153615975, 1.000000000000001, 0.0823760818974285]
[0.152575318863385, 0.09084068775666247, 0.16387233403063464, 0.148507501475717, 0.0823760818974285, 1.0000000000000016]


## P4- Similitud de coseno con el Indice Invertido

### 1- Estructura del índice invertido en Python:

In [3]:
"""
index = {
w1 : [(doc1, tf_w1_doc1), (doc3, tf_w1_doc3),(doc4, tf_w1_doc4),(doc10, tf_w1_doc10)],
w2 : [(doc1, tf_w2_doc1 ), (doc2, tf_w2_doc2)],
w3 : [(doc2, tf_w3_doc2), (doc3, tf_w3_doc3),(doc7, tf_w3_doc7)],
}

idf = {
w1 : idf_w1,
w2 : idf_w2,
w3 : idf_w3,
}

length ={
doc1: norm_doc1,
doc2: norm_doc2,
doc3: norm_doc3,
...
}
"""

'\nindex = {\nw1 : [(doc1, tf_w1_doc1), (doc3, tf_w1_doc3),(doc4, tf_w1_doc4),(doc10, tf_w1_doc10)],\nw2 : [(doc1, tf_w2_doc1 ), (doc2, tf_w2_doc2)],\nw3 : [(doc2, tf_w3_doc2), (doc3, tf_w3_doc3),(doc7, tf_w3_doc7)],\n}\n\nidf = {\nw1 : idf_w1,\nw2 : idf_w2,\nw3 : idf_w3,\n}\n\nlength ={\ndoc1: norm_doc1,\ndoc2: norm_doc2,\ndoc3: norm_doc3,\n...\n}\n'

### 2- Algoritmo para construir el índice
#### Paso 1:	Implementar los métodos de la clase

In [9]:
import math
import pickle
from collections import defaultdict

class InvertIndex:
    def __init__(self, index_file):
        self.index_file = index_file
        self.index = {}
        self.idf = {}
        self.length = {}
    
    def building(self, collection_text, position_text):
        num_documents = len(collection_text)
        term_doc_freq = defaultdict(int)
        doc_term_freq = defaultdict(lambda: defaultdict(int))
        
        for doc_id, text in enumerate(collection_text):
            terms = text.split()
            for term in terms:
                doc_term_freq[doc_id][term] += 1
                term_doc_freq[term] += 1

        for doc_id, terms_freq in doc_term_freq.items():
            self.length[doc_id] = 0
            for term, tf in terms_freq.items():
                tf_weight = 1 + math.log10(tf)
                idf_weight = math.log10(num_documents / term_doc_freq[term])
                
                self.idf[term] = idf_weight
                
                if term not in self.index:
                    self.index[term] = {}
                self.index[term][doc_id] = tf_weight * idf_weight
                
                self.length[doc_id] += (tf_weight * idf_weight) ** 2
        
        for doc_id in self.length:
            self.length[doc_id] = math.sqrt(self.length[doc_id])
        
        with open(self.index_file, 'wb') as f:
            pickle.dump((self.index, self.idf, self.length), f)
        
    def retrieval(self, query, k):
        self.load_index()
        score = defaultdict(float)
        
        query_terms = query.split()
        query_tf = defaultdict(int)
        
        for term in query_terms:
            query_tf[term] += 1
        
        query_weights = {}
        query_length = 0
        for term, tf in query_tf.items():
            tf_weight = 1 + math.log10(tf)
            idf_weight = self.idf.get(term, 0)
            query_weights[term] = tf_weight * idf_weight
            query_length += (tf_weight * idf_weight) ** 2
        
        query_length = math.sqrt(query_length)
        
        for term, query_weight in query_weights.items():
            if term in self.index:
                for doc_id, doc_weight in self.index[term].items():
                    score[doc_id] += query_weight * doc_weight
        
        for doc_id in score:
            if self.length[doc_id] != 0 and query_length != 0:
                score[doc_id] /= (self.length[doc_id] * query_length)

        result = sorted(score.items(), key=lambda tup: tup[1], reverse=True)
        return result[:k]

    def load_index(self):
        with open(self.index_file, 'rb') as f:
            self.index, self.idf, self.length = pickle.load(f)


### Utilizar el siguiente algoritmo. Asegúrese que la similitud de dos documentos iguales sea 1. 

<img src="image-20241004-152440.png" width="75%" align="" />

### Paso 2:	Probar el Índice

In [10]:
import pandas as pd

# Utilizar la siguiente coleccion de documentos para probar la eficiencia del indice
dataton = pd.read_csv('df_total.csv')
dataton.head()

,url,news,Type
0,https://www.larepublica.co/redirect/post/3201905,Durante el foro La banca articulador empresari...,Otra
1,https://www.larepublica.co/redirect/post/3210288,El regulador de valores de China dijo el domin...,Regulaciones
2,https://www.larepublica.co/redirect/post/3240676,En una industria históricamente masculina como...,Alianzas
3,https://www.larepublica.co/redirect/post/3342889,Con el dato de marzo el IPC interanual encaden...,Macroeconomia
4,https://www.larepublica.co/redirect/post/3427208,Ayer en Cartagena se dio inicio a la versión n...,Otra


In [11]:
def mostrarDocumentos(result, dataton):
    """Muestra los documentos relevantes basados en el resultado de la búsqueda"""
    print("Documentos más relevantes:\n")
    for doc_id, score in result:
        print(f"Documento {doc_id + 1}:")
        print(f"URL: {dataton.loc[doc_id, 'url']}")
        print(f"Texto: {dataton.loc[doc_id, 'news']}")
        print(f"Similitud (Coseno): {score:.4f}")
        print("-" * 40)

index = InvertIndex("indice.dat")
index.building(dataton['news'], 1)

Query1 = "El pais de China y su cooperacion"
result = index.retrieval(Query1, 10)

mostrarDocumentos(result, dataton)


Documentos más relevantes:

Documento 804:
URL: https://www.larepublica.co/redirect/post/3281318
Texto: El Consumption Tracker es un indicador que construye Bbva haciendo uso de las transacciones realizadas por sus clientes con tarjetas débito y crédito. Los datos de la última edición tienen cierre al 21 de diciembre de 2021.La medición destacó que en lo corrido de diciembre repuntó el gasto en entretenimiento que por primera vez desde el inicio de la pandemia se ubicó por encima de los niveles registrados en el mismo periodo de 2019.En Bogotá el consumo en lo corrido del mes creció 144 frente a 2019 y se desaceleró frente al dato de noviembre 215.
Similitud (Coseno): 0.2650
----------------------------------------
Documento 829:
URL: https://www.larepublica.co/redirect/post/3318399
Texto: En medio de la apuesta de Bbva por la transformación digital de la banca la entidad financiera en Colombia ha aumentado su talento humano femenino. De hecho en el área de tecnología e ingeniería la c

In [12]:
Query2 = "Tecnología y educación en la universidad"
result2 = index.retrieval(Query2, 10)
mostrarDocumentos(result2, dataton)

Documentos más relevantes:

Documento 910:
URL: https://www.larepublica.co/redirect/post/3400214
Texto: Los cambios tecnológicos modifican la forma en que se relacionan las personas en todas las áreas. La educación no es la excepción puesto que las nuevas generaciones buscan acceder a contenidos y cursos en línea muy específicos a los que ingresan a través de diferentes plataformas con profesores profesionales de diversas áreas en un tiempo mucho más corto que en una universidad centro de estudios o institución formativa.Por ello LR le cuenta dos plataformas en las que obtiene un certificado express a un costo asequible para las personas que buscan formación concisa y rápida.La primera de estas plataformas es Crehana que actualmente cuenta con más de 1.000 cursos online y que plantea un modelo educativo por resultados. A su turno una de las más conocidas entre los internautas es Platzi compañía de educación en línea en las áreas de mayor demanda laboral y que ayuda a desarrollar el per

In [13]:
Query3 = "Electricidad y desarrollo internacional"
result3 = index.retrieval(Query3, 10)
mostrarDocumentos(result3, dataton)

Documentos más relevantes:

Documento 981:
URL: https://www.larepublica.co/redirect/post/3167349
Texto: El primer insumo de cara a la creación del marco regulatorio del hidrógeno verde en Chile pondrá el gobierno este lunes sobre la mesa. En una resolución exenta las autoridades revelarán la denominada guía de proyectos especiales en este elemento para responder al interés que ha surgido en el país por emprender iniciativas en esta industria que recién empieza a despegar.Actualmente las solicitudes para emprender proyectos de este tipo son presentadas a la Superintendencia de Electricidad y Combustibles SEC pero según la industria no existen lineamientos claros de esta revisión. Fuentes comentan que la entidad no tiene un instructivo que indique qué es necesario o cómo se tiene que tramitar los proyectos especiales. En sí este concepto sólo está en una norma de un decreto supremo que enuncia algunos temas que deben incluirse. Por esto sería clave la guía ahora que el interés por esta i

In [14]:
Query4 = "Influencia del clima en la sociedad"
result4 = index.retrieval(Query4, 10)
mostrarDocumentos(result4, dataton)

Documentos más relevantes:

Documento 649:
URL: https://www.bbva.com/es/mx/banxico-podria-aprovechar-avances-en-transparencia-para-combatir-la-inflacion/
Texto: El Banco de México enfrenta una etapa de importantes retos derivada del impacto que la alta inflación ha tenido en las economías del mundo. De acuerdo con Arnulfo Rodríguez Hernández economista principal de BBVA México la transparencia en la toma de decisiones del banco central juega un papel muy relevante para mantener su credibilidad. En su artículo La comunicación de Banxico avances y retos publicado en el periódico El Economista Rodríguez menciona que desde abril de 1994 se le otorgó autonomía al Banco de México a través de una reforma constitucional decisión que ha permitido cumplir con el mandato más importante que tiene que es mantener el poder adquisitivo de la moneda nacional precepto que cobra mayor relevancia al estar vinculado con la rendición de cuentas a la sociedad sobre las decisiones que tome y donde la transpa

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=b824a2a4-ec75-4e0c-9c8d-ae3f8e552746' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>